In [1]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import pybaseball
from pybaseball import *
from nameparser import HumanName
from sklearn.preprocessing import MinMaxScaler
from lxml import html

In [2]:
#playerid_lookup('Judge')

In [3]:
# URL of the webpage containing the HTML with tables
url = "https://www.foxsports.com/articles/mlb/2024-mlb-mvp-odds-al-and-nl-favorites"

# Fetch the HTML content of the webpage
response = requests.get(url)
html_content = response.text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [4]:
# Find all table elements in the HTML
tables = soup.find_all("table")

# Iterate over each table
for index, table in enumerate(tables, 1):
    # Create a filename for the CSV file
    filename = f"table_{index}.csv"

    # Open the CSV file for writing
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)

        # Find table header (thead) and write column headers to CSV
        headers = [header.get_text(strip=True) for header in table.find("thead").find_all("th")]
        writer.writerow(headers)

        # Find table rows (tbody) and write row data to CSV
        for row in table.find("tbody").find_all("tr"):
            row_data = [data.get_text(strip=True) for data in row.find_all(["th", "td"])]
            writer.writerow(row_data)

    print(f"Table {index} saved to {filename}.")

Table 1 saved to table_1.csv.
Table 2 saved to table_2.csv.


In [5]:
path = 'C:/Users/kenne/mvp_analysis/'
file_path = "table_1.csv"
file_path_2 = "table_2.csv"
df = pd.read_csv(file_path)
df_2 = pd.read_csv(file_path_2)

In [6]:
al_mvp_candidates = df['Name'].tolist()
nl_mvp_candidates = df_2['Name'].tolist()

In [7]:
def remove_jr(name):
    return name.replace(" Jr.", "") if " Jr." in name else name.replace("Jr.", "")

# Apply the function to each candidate list
al_mvp_candidates = [remove_jr(name) for name in al_mvp_candidates]
nl_mvp_candidates = [remove_jr(name) for name in nl_mvp_candidates]

print(al_mvp_candidates)
print(nl_mvp_candidates)

['Aaron Judge', 'Juan Soto', 'Gunnar Henderson', 'Bobby Witt', 'José Ramírez', 'Kyle Tucker', 'Corey Seager', 'Steven Kwan', 'Adley Rutschman', 'Yordan Alvarez', 'Rafael Devers', 'Jose Altuve', 'Marcus Semien', 'Salvador Pérez', 'Anthony Volpe', "Tyler O'Neill", 'Ryan Jeffers', 'Adolis García', 'Colton Cowser', 'Riley Greene']
['Shohei Ohtani', 'Bryce Harper', 'Marcell Ozuna', 'Freddie Freeman', 'Fernando Tatis', 'Ketel Marte', 'William Contreras', 'Matt Olson', 'Mookie Betts', 'Alec Bohm', 'Christian Yelich', 'Francisco Lindor', 'Elly De La Cruz', 'C.J. Abrams', 'Luis Arraez', 'Pete Alonso', 'Zack Wheeler', 'Austin Riley', 'Willy Adames', 'Christian Walker']


In [8]:
all_candidates = [al_mvp_candidates, nl_mvp_candidates]


# Define a function to get player information and write it to a CSV file
def get_player_info_and_write_to_csv(candidate_list, league):
    all_player_info = []
    for name in candidate_list:
        name_parts = HumanName(name)
        last = name_parts.last
        first = name_parts.first
        data = playerid_lookup(last, first, fuzzy=True)
        if not data.empty:  # Check if the DataFrame is not empty
            all_player_info.append(data)  # Append the DataFrame if not empty

    if all_player_info:
        # Concatenate the collected DataFrames into a single DataFrame
        all_player_info_df = pd.concat(all_player_info)
        
        # Write the player information to a CSV file
        file_name = f"{league}_player_info.csv"
        all_player_info_df.to_csv(file_name, index=False)

# Get player information and write to CSV for AL candidates
get_player_info_and_write_to_csv(al_mvp_candidates, "AL")

# Get player information and write to CSV for NL candidates
get_player_info_and_write_to_csv(nl_mvp_candidates, "NL")

Gathering player lookup table. This may take a moment.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.


In [9]:
path = 'C:/Users/kenne/mvp_analysis/'
file_path = "AL_player_info.csv"
file_path_2 = "NL_player_info.csv"
df = pd.read_csv(file_path)
df_2 = pd.read_csv(file_path_2)

In [10]:
df.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,judge,aaron,592450,judga001,judgeaa01,15640,2016.0,2024.0
1,soto,juan,665742,sotoj001,sotoju01,20123,2018.0,2024.0
2,henderson,gunnar,683002,hendg002,hendegu01,26289,2022.0,2024.0
3,witt,bobby,677951,wittb002,wittbo02,25764,2022.0,2024.0
4,witt,bobby,124492,wittb001,wittbo01,1014170,1986.0,2001.0


In [11]:
df_2.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,ohtani,shohei,660271,ohtas001,ohtansh01,19755,2018.0,2024.0
1,harper,bryce,547180,harpb003,harpebr03,11579,2012.0,2024.0
2,ozuna,marcell,542303,ozunm001,ozunama01,10324,2013.0,2024.0
3,freeman,freddie,518692,freef001,freemfr01,5361,2010.0,2024.0
4,tatís,fernando,665487,tatif002,tatisfe02,19709,2019.0,2024.0


In [12]:
df = df[df['mlb_played_last'] == 2024]

# Display the filtered DataFrame
print(df)
df.shape

    name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0       judge      aaron     592450  judga001  judgeaa01          15640   
1        soto       juan     665742  sotoj001   sotoju01          20123   
2   henderson     gunnar     683002  hendg002  hendegu01          26289   
3        witt      bobby     677951  wittb002   wittbo02          25764   
6     ramírez       josé     608070  ramij003  ramirjo01          13510   
7      tucker       kyle     663656  tuckk001  tuckeky01          18345   
8      seager      corey     608369  seagc001  seageco01          13624   
9        kwan     steven     680757  kwans001   kwanst01          24610   
10  rutschman      adley     668939  rutsa001  rutscad01          26288   
11    álvarez     yordan     670541  alvay001  alvaryo01          19556   
16     devers     rafael     646240  dever001  deverra01          17350   
17     altuve       josé     514888  altuj001  altuvjo01           5417   
22     semien     marcus 

(20, 8)

In [13]:
df_2 = df_2[df_2['mlb_played_last'] == 2024]

# Display the filtered DataFrame
print(df_2)
df_2.shape

     name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0       ohtani     shohei     660271  ohtas001  ohtansh01          19755   
1       harper      bryce     547180  harpb003  harpebr03          11579   
2        ozuna    marcell     542303  ozunm001  ozunama01          10324   
3      freeman    freddie     518692  freef001  freemfr01           5361   
4        tatís   fernando     665487  tatif002  tatisfe02          19709   
6        tatís   fernando     665487  tatif002  tatisfe02          19709   
11       marte      ketel     606466  martk001  marteke01          13613   
12   contreras    william     661388  contw002  contrwi02          20503   
13       olson       matt     621566  olsom001  olsonma02          14344   
14       betts     mookie     605141  bettm001  bettsmo01          13611   
15        bohm       alec     664761  bohma001   bohmal01          21618   
16      yelich  christian     592885  yelic001  yelicch01          11477   
17      lind

(21, 8)

In [14]:
df_2 = df_2.drop_duplicates()
print(df_2)
df_2.shape

     name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0       ohtani     shohei     660271  ohtas001  ohtansh01          19755   
1       harper      bryce     547180  harpb003  harpebr03          11579   
2        ozuna    marcell     542303  ozunm001  ozunama01          10324   
3      freeman    freddie     518692  freef001  freemfr01           5361   
4        tatís   fernando     665487  tatif002  tatisfe02          19709   
11       marte      ketel     606466  martk001  marteke01          13613   
12   contreras    william     661388  contw002  contrwi02          20503   
13       olson       matt     621566  olsom001  olsonma02          14344   
14       betts     mookie     605141  bettm001  bettsmo01          13611   
15        bohm       alec     664761  bohma001   bohmal01          21618   
16      yelich  christian     592885  yelic001  yelicch01          11477   
17      lindor  francisco     596019  lindf001  lindofr01          12916   
18  de la cr

(20, 8)

In [15]:
# Remove duplicates based on the 'name_last' column
df.drop_duplicates(subset='name_last', keep='first', inplace=True)

# Display the DataFrame after removing duplicates
print(df)
df.shape

    name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0       judge      aaron     592450  judga001  judgeaa01          15640   
1        soto       juan     665742  sotoj001   sotoju01          20123   
2   henderson     gunnar     683002  hendg002  hendegu01          26289   
3        witt      bobby     677951  wittb002   wittbo02          25764   
6     ramírez       josé     608070  ramij003  ramirjo01          13510   
7      tucker       kyle     663656  tuckk001  tuckeky01          18345   
8      seager      corey     608369  seagc001  seageco01          13624   
9        kwan     steven     680757  kwans001   kwanst01          24610   
10  rutschman      adley     668939  rutsa001  rutscad01          26288   
11    álvarez     yordan     670541  alvay001  alvaryo01          19556   
16     devers     rafael     646240  dever001  deverra01          17350   
17     altuve       josé     514888  altuj001  altuvjo01           5417   
22     semien     marcus 

(20, 8)

In [16]:
print(df_2)

     name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0       ohtani     shohei     660271  ohtas001  ohtansh01          19755   
1       harper      bryce     547180  harpb003  harpebr03          11579   
2        ozuna    marcell     542303  ozunm001  ozunama01          10324   
3      freeman    freddie     518692  freef001  freemfr01           5361   
4        tatís   fernando     665487  tatif002  tatisfe02          19709   
11       marte      ketel     606466  martk001  marteke01          13613   
12   contreras    william     661388  contw002  contrwi02          20503   
13       olson       matt     621566  olsom001  olsonma02          14344   
14       betts     mookie     605141  bettm001  bettsmo01          13611   
15        bohm       alec     664761  bohma001   bohmal01          21618   
16      yelich  christian     592885  yelic001  yelicch01          11477   
17      lindor  francisco     596019  lindf001  lindofr01          12916   
18  de la cr

In [17]:
playerIds_Al = df["key_fangraphs"]
print(playerIds_Al)

0     15640
1     20123
2     26289
3     25764
6     13510
7     18345
8     13624
9     24610
10    26288
11    19556
16    17350
17     5417
22    12533
23     7304
24    27647
25    15711
26    24618
27    19287
28    29591
29    25976
Name: key_fangraphs, dtype: int64


In [18]:
playerIds_NL = df_2["key_fangraphs"]
print(playerIds_NL)

0     19755
1     11579
2     10324
3      5361
4     19709
11    13613
12    20503
13    14344
14    13611
15    21618
16    11477
17    12916
18    26668
19    25768
24    18568
30    19251
31    10310
32    18360
33    15986
34    13419
Name: key_fangraphs, dtype: int64


In [19]:
stats = batting_stats(2024)

In [20]:
stats.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2024,Aaron Judge,NYY,32,74,265,328,80,31,...,116.1,116,0.624,186,0.150,0.277,0.303,0.734,0.474,5.1
1,20123,2024,Juan Soto,NYY,25,72,262,326,82,49,...,115.7,126,0.589,214,0.196,0.271,0.317,0.648,0.463,4.8
5,26289,2024,Gunnar Henderson,BAL,23,71,278,323,76,38,...,113.1,114,0.564,202,0.158,0.264,0.278,0.537,0.389,4.7
10,25764,2024,Bobby Witt Jr.,KCR,24,74,299,329,96,59,...,113.3,120,0.484,248,0.149,0.252,0.327,0.610,0.417,4.4
3,19755,2024,Shohei Ohtani,LAD,29,72,288,327,91,49,...,119.2,139,0.610,228,0.145,0.261,0.341,0.665,0.455,3.7


In [21]:
al_merged = pd.merge(df, stats, left_on='key_fangraphs', right_on='IDfg')
nl_merged = pd.merge(df_2, stats, left_on='key_fangraphs', right_on='IDfg')

In [22]:
print(al_merged)
print(nl_merged)

    name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0       judge      aaron     592450  judga001  judgeaa01          15640   
1        soto       juan     665742  sotoj001   sotoju01          20123   
2   henderson     gunnar     683002  hendg002  hendegu01          26289   
3        witt      bobby     677951  wittb002   wittbo02          25764   
4     ramírez       josé     608070  ramij003  ramirjo01          13510   
5      tucker       kyle     663656  tuckk001  tuckeky01          18345   
6      seager      corey     608369  seagc001  seageco01          13624   
7   rutschman      adley     668939  rutsa001  rutscad01          26288   
8     álvarez     yordan     670541  alvay001  alvaryo01          19556   
9      devers     rafael     646240  dever001  deverra01          17350   
10     altuve       josé     514888  altuj001  altuvjo01           5417   
11     semien     marcus     543760  semim001  semiema01          12533   
12      pérez   salvador 

In [23]:
def normalize_numeric_columns(df):
    if 'IDfg' in df.columns:
        idfg_col = df['IDfg']
    else:
        idfg_col = None
    
    # Select numeric columns to normalize
    numeric_cols = df.select_dtypes(include=['int', 'float']).columns
    
    # Initialize MinMaxScaler
    scaler = MinMaxScaler()
    
    # Apply MinMaxScaler to numeric columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Put 'idfg' column back into the DataFrame if it was extracted
    if idfg_col is not None:
        df['IDfg'] = idfg_col
    
    return df

# Normalize numeric columns
al_normalized = normalize_numeric_columns(al_merged)
nl_normalized = normalize_numeric_columns(nl_merged)

al_normalized.head()

C:\Users\kenne\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:696: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\kenne\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:713: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\kenne\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:696: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\kenne\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:713: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,IDfg,Season,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,judge,aaron,0.461341,judga001,judgeaa01,0.422892,0.416667,0.0,15640,0.0,...,0.929293,0.871795,1.000000,0.456140,0.500000,0.663717,0.766990,1.000000,1.000000,1.00
1,soto,juan,0.897284,sotoj001,sotoju01,0.608340,0.583333,0.0,20123,0.0,...,0.888889,1.000000,0.892638,0.701754,0.925926,0.610619,0.902913,0.774278,0.938547,0.94
2,henderson,gunnar,0.999946,hendg002,hendegu01,0.863407,0.916667,0.0,26289,0.0,...,0.626263,0.846154,0.815951,0.596491,0.574074,0.548673,0.524272,0.482940,0.525140,0.92
3,witt,bobby,0.969903,wittb002,wittbo02,0.841689,0.916667,0.0,25764,0.0,...,0.646465,0.923077,0.570552,1.000000,0.490741,0.442478,1.000000,0.674541,0.681564,0.86
4,ramírez,josé,0.554249,ramij003,ramirjo01,0.334781,0.166667,0.0,13510,0.0,...,0.212121,0.564103,0.276074,0.903509,0.407407,0.070796,0.155340,0.188976,0.094972,0.46


In [25]:
nl_normalized.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,IDfg,Season,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,ohtani,shohei,0.862046,ohtas001,ohtansh01,0.675553,0.615385,0.0,19755,0.0,...,1.000000,1.000000,1.000000,0.488,0.386792,0.476636,1.000000,1.000000,1.000,1.000000
1,harper,bryce,0.173458,harpb003,harpebr03,0.291829,0.153846,0.0,11579,0.0,...,0.379630,0.333333,0.649616,0.168,0.000000,0.308411,0.390909,0.368821,0.352,0.633333
2,ozuna,marcell,0.143763,ozunm001,ozunama01,0.232928,0.230769,0.0,10324,0.0,...,0.574074,0.626667,0.846547,0.280,0.245283,0.551402,0.763636,0.980989,0.872,0.733333
3,freeman,freddie,0.000000,freef001,freemfr01,0.000000,0.000000,0.0,5361,0.0,...,0.361111,0.306667,0.432225,0.456,0.150943,0.000000,0.454545,0.304183,0.408,0.700000
4,tatís,fernando,0.893805,tatif002,tatisfe02,0.673394,0.692308,0.0,19709,0.0,...,0.777778,0.746667,0.785166,0.488,0.235849,0.373832,0.627273,0.528517,0.496,0.566667


In [26]:
al_normalized.to_csv("AL_prospects.csv")
nl_normalized.to_csv("NL_prospects.csv")